In [39]:
# transformers not support NumPy 2.0 yet
!pip install -q numpy~=1.26.4 transformers~=4.46.2
!pip install -q datasets evaluate pydantic seqeval

# 訓練 PII 遮掩模型

In [40]:
import pandas as pd
import numpy as np
import evaluate

from transformers import (
  AutoTokenizer,
  AutoModelForTokenClassification,
  TrainingArguments,
  Trainer,
)
from datasets import load_dataset, DatasetDict
from transformers import DataCollatorForTokenClassification
from transformers import pipeline

from pydantic import BaseModel
from pprint import pprint

import ast
import torch

# 檢查是否有 GPU 可以使用
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = torch.device("mps" if torch.backends.mps.is_available() else device)

## 下載資料

In [55]:
# The full `train` split
immutable_dataset = load_dataset("ai4privacy/pii-masking-65k", split="train")

## 資料包含什麼？

In [ ]:
# Reserve 0.05% of the training set for testing
test_dataset = immutable_dataset.train_test_split(
  test_size=0.0005, # 0.05% of the data is used for testing
  shuffle=False, # Ensure that train and validation sets are the same across runs
  )
# Split into 80% training and 20% validation sets
train_dataset = test_dataset['train'].train_test_split(
  test_size=0.2, # 20% of the data is used for validation
  shuffle=False, # Ensure that train and test sets are the same across runs
  )
immutable_dataset = DatasetDict({
  'train': train_dataset['train'],
  'validation': train_dataset['test'],
  'test': test_dataset['test'],
  })
immutable_dataset
